Generating and Testing the Fried Infinate Phase Screens
=======================================================

In [ ]:
import numpy
import matplotlib.pylab as plt
from tqdm import tqdm, trange, tqdm_notebook
from astropy.io import fits

import aotools
from aotools.turbulence.infinitephasescreen import PhaseScreenKolmogorov
from aotools.turbulence import infinitephasescreen

In [ ]:
# Set up parameters for creating phase screens
n_zerns = 60
nx_size = 64
D = 1.
pxl_scale = D/nx_size
r0 = 0.164
L0 = 100
wind_speed = 10 #m/s - just arbitrarily set
n_tests = 25 # 16
n_scrns = 100
noll = fits.getdata("../noll.fits").diagonal() * (D/r0)**(5./3)
stencil_length_factor = 32

Zs = aotools.zernikeArray(n_zerns, nx_size)

Example Phase Screen
--------------------

In [ ]:
phase_screen = PhaseScreenKolmogorov(nx_size, pxl_scale, r0, L0, stencil_length_factor=stencil_length_factor)
print(phase_screen.scrn.shape)
plt.figure()
plt.imshow(phase_screen.scrn)
cbar = plt.colorbar()
cbar.set_label('Wavefront deviation (radians)', labelpad=8)

Zernike Breakdown of Phase Screen
---------------------------------

In [ ]:
time_step = pxl_scale/wind_speed # This is timestep, as one pixel added on each iteration

z_coeffs_inf_f = numpy.zeros((n_tests, n_scrns, n_zerns))

for n in tqdm(range(n_tests)):
    phase_screen = PhaseScreenKolmogorov(nx_size, pxl_scale, r0, L0, stencil_length_factor=stencil_length_factor)
    for i in range(n_scrns):
        # Go in all directions
        phase_screen.add_row()
        if n%4 == 0:
            scrn = phase_screen.scrn
        elif n%4 == 1:
            scrn = phase_screen.scrn.T
        elif n%4 == 2:
            scrn = phase_screen.scrn[::-1]
        else:
            scrn = phase_screen.scrn[::-1].T
            
        z_coeffs_inf_f[n, i] = (scrn * Zs).sum((-1, -2))/(Zs[0].sum())
        
z_vars_inf_f = z_coeffs_inf_f.var(1)

In [ ]:
f, a = plt.subplots()
a.plot(z_vars_inf_f.mean(0), color="b", label="Phase Screen")
a.plot(range(1, n_zerns+1), noll[:n_zerns] , label="Noll", color="k")
a.set_yscale('log')
a.set_xlabel("Zernike Index")
a.set_ylabel("Power ($\mathrm{RMS Rad}^2$)")
plt.legend()

r<sub>0</sub> Scaling
---------------------

In [ ]:
r0s = [0.05, 0.1, 0.15, 0.2]
screens = 1
size = 256
region = 128
D = 4.2
pxl_scale = float(D/region)

for i, r0 in enumerate(r0s):
    for j in range(screens):
        phase_screen = infinitephasescreen.PhaseScreen(size, pxl_scale, r0, 100., stencil_length_factor=stencil_length_factor)
        
        region_screen = phase_screen.scrn[100:100+region, 100:100+region]
        variance = region_screen.var()
        print(r0, variance)

In [ ]:
from aotools.turbulence import ft_phase_screen

In [ ]:
infinitephasescreen.PhaseScreen.